In [1]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import geopandas as gpd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

import yaml

from paths import *
from scripts import manipulation
from scripts import io

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
from scripts import vis_layout

from os import listdir
import glob


##  Portal da Transparência | Organizações Sociais de Saúde - OSS

 - [Portal da Transparência OSS](http://www.portaldatransparencia.saude.sp.gov.br/index.php)

In [2]:
geo_sp = gpd.read_file('../data/geodata/sp_municipios.json')

In [3]:
#oss qualificadas
quali = pd.concat(map(pd.read_csv, glob.glob('../data/oss_qualificadas/years/*.csv')))
quali_orgs = pd.concat(map(pd.read_csv, glob.glob('../data/oss_qualificadas/organizacoes/*.csv')))

#oss contratadas
contrat = pd.concat(map(pd.read_csv, glob.glob('../data/oss_contratualizadas/years/*.csv')))
contrat_org = pd.concat(map(pd.read_csv, glob.glob('../data/oss_contratualizadas/organizacoes/*.csv')))

# oss kpis
indicadores = pd.concat(map(pd.read_csv, glob.glob('../data/oss_indicadores/*.csv')))
indicadores = indicadores[indicadores['Unidade']!='Unidade']

#oss convocadas
convocacoes = pd.concat(map(pd.read_csv, glob.glob('../data/oss_convocacoes/*.csv')))

## meso region data
cols = ['geocodigo','nome_uf','nome_municipio','nome_mesorregiao']
sp = geo_sp[cols]
sp['Municipio'] = sp['nome_municipio']
sp['nome_municipio']          = sp['nome_municipio'].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

#oss and cities data
municipios_quali = pd.concat(map(pd.read_csv, glob.glob('../data/oss_cidades/*.csv')))
municipios_quali = municipios_quali.reset_index(drop=True)
municipios_quali['municipio'] = municipios_quali['municipio'].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
municipios_quali = municipios_quali.merge(sp, left_on='municipio', right_on='nome_municipio',how='left')

#oss with cities
oss = municipios_quali.sort_values(by='Ano', ascending=False).drop_duplicates(subset = ['oss','unidade'], keep='first').reset_index(drop=True).sort_values(by='oss', ascending=True).reset_index(drop=True)
cols = ['oss','unidade','cnes','cnpj','geocodigo','nome_uf','Municipio','endereco','bairro','nome_mesorregiao']
oss = oss[cols]

In [4]:
oss['nome_mesorregiao'].value_counts()

Metropolitana de São Paulo      78
Macro Metropolitana Paulista    11
São José do Rio Preto           10
Ribeirão Preto                   8
Bauru                            8
Vale do Paraíba Paulista         7
Litoral Sul Paulista             6
Presidente Prudente              4
Araçatuba                        3
Itapetininga                     3
Assis                            3
Campinas                         3
Araraquara                       2
Marília                          1
Name: nome_mesorregiao, dtype: int64

In [15]:
mask = oss['nome_mesorregiao']=='Vale do Paraíba Paulista'
oss[mask]

,oss,unidade,cnes,cnpj,geocodigo,nome_uf,Municipio,endereco,bairro,nome_mesorregiao
19,Associação Paulista para o Desenvolvimento da Medicina - SPDM,Ambulatório Médico de Especialidades São José dos Campos(AME São José dos Campos),6294049.0,61.699.567/0022-17,3549904,São Paulo,São José dos Campos,"Av. Engenheiro Francisco José Longo, 925",Jd. São Dimas,Vale do Paraíba Paulista
24,Associação Paulista para o Desenvolvimento da Medicina - SPDM,Centro de Medicina de Reabilitação Lucy Montoro de São José dos Campos(Lucy Montoro de São José dos Campos),6889549.0,61.699.567/0036-12,3549904,São Paulo,São José dos Campos,"Saigiro Nakamura, 600",Centro,Vale do Paraíba Paulista
96,Instituto Sócrates Guanaes - ISG,Hospital Regional de São José dos Campos(Hospital Regional de São José dos Campos),9491252.0,NaN,3549904,São Paulo,São José dos Campos,"Avenida Fortaleza, SN",Parque Industrial,Vale do Paraíba Paulista
100,Instituto Sócrates Guanaes - ISG,Ambulatório Médico de Especialidades São José dos Campos(AME São José dos Campos),6294049.0,61.699.567/0022-17,3549904,São Paulo,São José dos Campos,"Av. Engenheiro Francisco José Longo, 925",Jd. São Dimas,Vale do Paraíba Paulista
137,Serviço Social da Construção Civil do Estado de São Paulo - SECONCI,Ambulatório Médico de Especialidades Caraguatatuba (AME Caraguatatuba),6233848.0,46.543.781/0012-14,3510500,São Paulo,Caraguatatuba,"Av. Acre, 1081",Indaia,Vale do Paraíba Paulista
141,Serviço Social da Construção Civil do Estado de São Paulo - SECONCI,Ambulatório Médico de Especialidades de Lorena(AME Lorena),9030557.0,NaN,3527207,São Paulo,Lorena,"Rua Dom Bosco, 562",Centro,Vale do Paraíba Paulista
146,Sociedade Assistencial Bandeirantes,Ambulatório Médico de Especialidades Caraguatatuba (AME Caraguatatuba),6233848.0,46.543.781/0012-14,3510500,São Paulo,Caraguatatuba,"Av. Acre, 1081",Indaia,Vale do Paraíba Paulista


In [24]:
oss[mask].loc[[19]].T

,19
oss,Associação Paulista para o Desenvolvimento da Medicina - SPDM
unidade,Ambulatório Médico de Especialidades São José dos Campos(AME São José dos Campos)
cnes,6.29405e+06
cnpj,61.699.567/0022-17
geocodigo,3549904
nome_uf,São Paulo
Municipio,São José dos Campos
endereco,"Av. Engenheiro Francisco José Longo, 925"
bairro,Jd. São Dimas
nome_mesorregiao,Vale do Paraíba Paulista


In [ ]:
# municipios_quali[mask].head()

In [ ]:
municipios_quali["Tipo"].unique()

## Portal Financeiro do Gestor
 - [Portal Financeiro do Gestor](http://www.portalfinanceirodogestor.saude.sp.gov.br/)

In [6]:
convenios =  pd.concat(map(pd.read_csv, glob.glob('../data/portal_financeiro/convenio/*.csv')))

fundo =  pd.concat(map(pd.read_csv, glob.glob('../data/portal_financeiro/fundo/*.csv')))

proprio =  pd.concat(map(pd.read_csv, glob.glob('../data/portal_financeiro/proprios/*.csv')))


oss_valores =  pd.concat(map(pd.read_csv, glob.glob('../data/portal_financeiro/oss/ano/*.csv')))
# oss_valores['OSS'] = oss_valores['OSS'].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [7]:
mask = oss_valores['ano'] == 2018
oss_valores[mask].sort_values(by=['Pago'], ascending=False)

,OSS,Gerenciadora atual,Contratado,Pago,ano
0,Hospital Santo André,Fundação ABC,1990.0000,2033.0000,2018
1,Hospital Guarulhos,SPDM,1831.0000,1820.0000,2018
2,Hospital Luzia de Pinho Melo,SPDM,1770.0000,1785.0000,2018
3,Hospital Bauru,FAMESP,1731.0000,1728.0000,2018
4,Hospital Presidente Prudente,Lar São Francisco,1668.0000,1680.0000,2018
5,Hospital Brigadeiro,SPDM,1573.0000,1597.0000,2018
6,Hospital Grajau,Sirío Libanês,1452.0000,1476.0000,2018
7,Hospital Pedreira,SPDM,1445.0000,1449.0000,2018
8,Hospital Pirajussara,SPDM,1300.0000,1365.0000,2018
9,Hospital Local Sapopemba,SECONCI,1350.0000,1341.0000,2018
